# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-b28f391fc8-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("F1 pilots explorative search") 

Consider the following exploratory information need:

> explore F1 pilots, their nationality, racing teams, and the country of the parent organization of the team

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P17`     | country       | predicate |
| `wd:Q10497835`| F1 Team       | node |
| `wd:Q10841764`| F1 Driver       | node |
| `wd:Q9671`    | Michael Schumacher      | node |
| `wd:Q169898`  | Scuderia Ferrari         | node |
| `wd:Q27586`   |  Ferrari         | node |
| `wd:Q183`     | Germany       | node |


Also consider

```
?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .
```

is the BGP to retrieve all **citizens of Germany**

```
?p wdt:P17 wd:Q183 .
?p wdt:P31 wd:Q10497835 .

``` 

is the BGP to retrieve all **F1 teams of Germany**

## Workload Goals

1. Identify the BGP for obtaining the team of a given  F1 driver  and also the F1 competitions in which they participated

2. Identify the BGP that connects a F1 team to a parent company

3. How many F1 drivers and F1 teams exist in each country?

4. Compare number of F1 drivers to their nationality
 
   4.1 Is there some driver that joined a team in their own country?
   
   4.2 Which team employed the larger number of drivers?
   
   4.3 For each country , which are the teams and drivers with the highest number of F1 race participations


In [2]:
# start your workflow here

## Identify the BGP for obtaining the team of a given F1 driver and also the F1 competitions in which they participated

I can start looking for the properties of Michael Schumacher

In [3]:
queryString = """
select distinct ?p ?pName ?o ?oName where {
    wd:Q9671 ?p ?o .
    
    optional { ?p <http://schema.org/name> ?pName . }
    optional { ?o <http://schema.org/name> ?oName . }
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language'), ('o', 'http://www.wikidata.org/entity/Q188'), ('oName', 'German')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10841764'), ('oName', 'Formula One driver')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q193391'), ('oName', 'diplomat')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q3014296'), ('oName', 'motorcycle rider')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q740308'), ('oName', 'UNICEF')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q67311526'), ('oName', 'Obalky knih.cz')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 

145

The property *member of sports team (**P54**)* is the desired one about the F1 teams the driver has. For what concerns the competitions the driver has partecipated, I can rely on the property *partecipant in (**P1344**)*. Finally, the desired BGP for F1 team is:

```
?driver wdt:P54 ?team
```

Since the competions can also be part of other leagues than F1, I can check the property for a F1 competion to look for something that connect the competion to F1.

In [4]:
queryString = """
select distinct ?p ?pName ?o ?oName where {
    wd:Q69713 ?p ?o .
    
    optional { ?p <http://schema.org/name> ?pName . }
    optional { ?o <http://schema.org/name> ?oName . }
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q171337'), ('oName', 'Williams Grand Prix Engineering')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q9671'), ('oName', 'Michael Schumacher')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q27020041'), ('oName', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition'), ('o', 'http://www.wikidata.org/entity/Q1968'), ('oName', 'Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5386'), ('oName', 'auto racing')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q179412'), ('oName', "Fédération Internationale de l'Automobile")]
[('p', '

26

So, the F1 competions are connected to *F1 (**Q1968**)* through the property *sports season of leauge or competion (**P3450**)*. The final BGP to get the F1 competions of a F1 driver is:

```
?driver wdt:P1344 ?competition .
?competition wdt:P3450 wd:Q1968 .
```

## 2. Identify the BGP that connects a F1 team to a parent company

I can start looking for the properties of the Scuderia Ferrari

In [5]:
queryString = """
select distinct ?p ?pName ?o ?oName where {
    wd:Q169898 ?p ?o .
    
    optional { ?p <http://schema.org/name> ?pName . }
    optional { ?o <http://schema.org/name> ?oName . }
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q10497835'), ('oName', 'Formula One team')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q20074337'), ('oName', 'auto racing team')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('pName', 'subsidiary'), ('o', 'http://www.wikidata.org/entity/Q5445262'), ('oName', 'Ferrari Driver Academy')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5386'), ('oName', 'auto racing')]
[('p', 'http://www.wikidata.org/prop/direct/P749'), ('pName', 'parent organization'), ('o', 'http://www.wikidata.org/entity/Q27586'), ('oName', 'Ferrari')]
[('p', 'http://www.wikidata.org/prop/direct/P859'), ('pName', 'sponsor'

42

And I can also look for the incoming properties

In [6]:
queryString = """
select distinct ?p ?pName ?o ?oName where {
    ?o ?p wd:Q169898 .
    
    optional { ?p <http://schema.org/name> ?pName . }
    optional { ?o <http://schema.org/name> ?oName . }
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q2835947'), ('oName', 'Ben Agathangelou')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q7107447'), ('oName', 'Ossi Oikarinen')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q3165116'), ('oName', 'Jean-Claude Migeot')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q173631'), ('oName', 'Pat Fry')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q19788193'), ('oName', 'Mattia Binotto')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q173692'), ('oName', 'Loïc Bigois')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')

67

The *parent company (**P749**)* and *subsidiary (**P355**)* are the properties I need. The desired BGP is:

```
?team ^wdt:P355|wdt:P749 ?parentCompany
```

I can list all the teams and their parents organizations, if any, considering that, as seen from the list of Ferrari's properties, F1 teams are instances of *F1 Team (**Q10497835**)*

In [7]:
queryString = """
select ?team ?teamName (group_concat(distinct ?parentName; separator=", ") as ?parentCompanies) where {
    ?team wdt:P31 wd:Q10497835 .
    
    optional { ?team ^wdt:P355|wdt:P749 ?parentCompany . ?parentCompany <http://schema.org/name> ?parentName } .
    
    ?team <http://schema.org/name> ?teamName .
} group by ?team ?teamName
order by desc(?parentCompanies)
"""

print("Results")
run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q173314'), ('teamName', 'Enzo Coloni Racing Car Systems'), ('parentCompanies', 'Scuderia Coloni')]
[('team', 'http://www.wikidata.org/entity/Q172030'), ('teamName', 'McLaren'), ('parentCompanies', 'McLaren Technology Group')]
[('team', 'http://www.wikidata.org/entity/Q173179'), ('teamName', 'Marussia F1'), ('parentCompanies', 'Marussia Motors')]
[('team', 'http://www.wikidata.org/entity/Q169898'), ('teamName', 'Scuderia Ferrari'), ('parentCompanies', 'Ferrari')]
[('team', 'http://www.wikidata.org/entity/Q2704273'), ('teamName', 'Aston Martin F1 Team'), ('parentCompanies', 'Aston Martin Lagonda')]
[('team', 'http://www.wikidata.org/entity/Q173295'), ('teamName', 'Modena'), ('parentCompanies', '')]
[('team', 'http://www.wikidata.org/entity/Q174008'), ('teamName', 'Caterham F1'), ('parentCompanies', '')]
[('team', 'http://www.wikidata.org/entity/Q842690'), ('teamName', 'Porsche'), ('parentCompanies', '')]
[('team', 'http://www.wikidata.org

67

## 3. How many F1 drivers and F1 teams exist in each country?

For what concern F1 drivers, I can count the number of drivers for each country.

In [8]:
queryString = """
select ?country ?countryName (count(distinct ?driver) as ?numDrivers) where {
    ?driver wdt:P106 wd:Q10841764 ;
            wdt:P27 ?country .
        
    ?country <http://schema.org/name> ?countryName .
} group by ?country ?countryName
order by desc(?numDrivers)
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('numDrivers', '245')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numDrivers', '172')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numDrivers', '58')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('numDrivers', '28')]
[('country', 'http://www.wikidata.org/entity/Q258'), ('countryName', 'South Africa'), ('numDrivers', '27')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('numDrivers', '25')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numDrivers', '19')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numDrivers', '18')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria'), ('numDrivers', '18')]
[('country', 'http://www.wikidata.org/ent

34

I see that there are some countries that are not existing anymore, such as West Germany or German Democratic Republic. It would be nice to merge these countries in the last one (in this case Germany). So firt, I look for the properties for West Germany (**Q713750**) and GDR (**Q16957**) to check if there is a path that connect these countries to Germany.

In [9]:
queryString = """
select distinct ?p ?pName where {
    values ?country { wd:Q713750 wd:Q16957 }
    ?country ?p wd:Q183 .
    
    ?p <http://schema.org/name> ?pName .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pName', 'diplomatic relation')]
[('p', 'http://www.wikidata.org/prop/direct/P3842'), ('pName', 'located in present-day administrative territorial entity')]


5

In [10]:
queryString = """
select distinct ?p ?pName where {
    values ?country { wd:Q713750 wd:Q16957 }
    wd:Q183 ?p ?country .
    
    ?p <http://schema.org/name> ?pName .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces')]


1

So there is a path coming from West Germany and GDR to Germany called *followed by (**P156**)* and *replaced by (**P1366**)*, and a property from Germany to West Germany/GDR called *replaces (**P1365**)*. I can write a BGP to obtain only the "last version" of the countries:

```
?country wdt:P156*|wdt:P1366*|^wdt:P1365* ?finalCountry .
```

So, I can rewrite the first query as follow:

In [11]:
queryString = """
select ?country ?countryName (count(distinct ?driver) as ?numDrivers) where {
    ?driver wdt:P106 wd:Q10841764 ;
            wdt:P27/(wdt:P156*|wdt:P1366*|^wdt:P1365*) ?country .
    
    filter not exists { ?country wdt:P156|wdt:P1366 ?o }
        
    ?country <http://schema.org/name> ?countryName .
} group by ?country ?countryName
order by desc(?numDrivers)
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('numDrivers', '245')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numDrivers', '172')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numDrivers', '62')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('numDrivers', '28')]
[('country', 'http://www.wikidata.org/entity/Q258'), ('countryName', 'South Africa'), ('numDrivers', '27')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('numDrivers', '25')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('countryName', 'Ireland'), ('numDrivers', '23')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numDrivers', '19')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia'), ('numDrivers', '19')]
[('country', 'http://www.wikidata.or

34

And I can do the same for the F1 teams.

In [12]:
queryString = """
select ?country ?countryName (count(distinct ?team) as ?numTeams) where {
    ?team wdt:P31 wd:Q10497835 ;
          wdt:P17/(wdt:P156*|wdt:P1366*|^wdt:P1365*) ?country .
    
    filter not exists { ?country wdt:P156|wdt:P1366 ?o }
    
    ?country <http://schema.org/name> ?countryName . 
} group by ?country ?countryName
order by desc(?numTeams)
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numTeams', '26')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numTeams', '16')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numTeams', '6')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('numTeams', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numTeams', '2')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan'), ('numTeams', '2')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia'), ('numTeams', '2')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countryName', 'Switzerland'), ('numTeams', '1')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia'), ('numTeams', '1')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('num

11

For what concerns the drivers, I considered the citizenship to assign a country to each driver. Another point of view can be considering the country of the driver's team, instead of the citizenship.

In [13]:
queryString = """
select ?country ?countryName (count(distinct ?driver) as ?numDrivers) where {
    ?driver wdt:P106 wd:Q10841764 ;
            wdt:P54 ?team .
    
    ?team wdt:P31 wd:Q10497835 ;
          wdt:P17/(wdt:P156*|wdt:P1366*|^wdt:P1365*) ?country .
    filter not exists { ?country wdt:P156|wdt:P1366 ?o }
    
    ?country <http://schema.org/name> ?countryName .
} group by ?country ?countryName
order by desc(?numDrivers)
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numDrivers', '55')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numDrivers', '33')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia'), ('numDrivers', '9')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numDrivers', '6')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countryName', 'Switzerland'), ('numDrivers', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numDrivers', '1')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan'), ('numDrivers', '1')]
[('country', 'http://www.wikidata.org/entity/Q833'), ('countryName', 'Malaysia'), ('numDrivers', '1')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia'), ('numDrivers', '1')]


9

## 4.1. Is there some driver that joined a team in their own country?

In order to answer to this question, I can select the drivers wich have same nationality and team's country

In [14]:
queryString = """
ask where {
    ?driver wdt:P106 wd:Q10841764 ;
            wdt:P27 ?country ;
            wdt:P54 ?team .
    
    ?team wdt:P31 wd:Q10497835 ;
          wdt:P17/(wdt:P156*|wdt:P1366*|^wdt:P1365*) ?country .
    filter not exists { ?country wdt:P156|wdt:P1366 ?o }
    
    ?country <http://schema.org/name> ?countryName .
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

There are some drivers that joined a team in their own country. I can also count how many drivers for country has the same nationality of their team's country, and compare to the total number of drivers of that country, to calculate a sort of "patriotism" level of the country's drivers.

In [15]:
queryString = """
select ?country ?countryName ?numTotalDrivers ?numPatrioticDrivers ?patriotismLevel where {
    {
        select ?originalCountry (count(distinct ?driver) as ?numTotalDrivers) where {
            ?driver wdt:P106 wd:Q10841764 ;
                    wdt:P27 ?originalCountry .
        }
    } .
    
    {
        select ?originalCountry (count(distinct ?driver) as ?numPatrioticDrivers) where {
            ?driver wdt:P106 wd:Q10841764 ;
                        wdt:P27 ?originalCountry ;
                        wdt:P54 ?team .

            ?team wdt:P31 wd:Q10497835 ;
                  wdt:P17 ?originalCountry .
        }
    }
    
    ?originalCountry wdt:P156*|wdt:P1366*|^wdt:P1365* ?country .
    filter not exists { ?originalCountry wdt:P156|wdt:P1366 ?o }
    ?country <http://schema.org/name> ?countryName .
    
    bind (xsd:int(?numPatrioticDrivers) * 100. / xsd:int(?numTotalDrivers) as ?patriotismLevel) .
    
} group by ?country ?countryName
order by desc(?patriotismLevel)
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia'), ('numTotalDrivers', '3'), ('numPatrioticDrivers', '1'), ('patriotismLevel', '33.333333333333333')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numTotalDrivers', '19'), ('numPatrioticDrivers', '6'), ('patriotismLevel', '31.578947368421053')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numTotalDrivers', '172'), ('numPatrioticDrivers', '20'), ('patriotismLevel', '11.627906976744186')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numTotalDrivers', '14'), ('numPatrioticDrivers', '1'), ('patriotismLevel', '7.142857142857143')]


4

## 4.2 Which team employed the larger number of drivers?

As done for the countries, I can simply count how many distinct drivers every team has, using the *member of sports team* property

In [16]:
queryString = """
select ?team ?teamName (count(distinct ?driver) as ?numDrivers) where {
    ?team wdt:P31 wd:Q10497835 .
    
    ?driver wdt:P106 wd:Q10841764 ;
            wdt:P54 ?team .
    
    ?team <http://schema.org/name> ?teamName .
} group by ?team ?teamName
order by desc(?numDrivers)
limit 1
"""

print("Results")
run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q169898'), ('teamName', 'Scuderia Ferrari'), ('numDrivers', '15')]


1

**Scuderia Ferrari** is the team that employed the larger number of drivers, with 15 drivers.

## 4.3. For each country , which are the teams and drivers with the highest number of F1 race participations

Start looking for the highest number of teams and drivers F1 race partecipations. For what concerns teams, I need to sum all the competitions of the drivers has partecipated in.

In [17]:
queryString = """
select ?countryName ?driver ?driverName (count(distinct ?competition) as ?numPartecipations) where {
    ?driver wdt:P106 wd:Q10841764 ;
            wdt:P27/(wdt:P156*|wdt:P1366*|^wdt:P1365*) ?country ;
            wdt:P1344 ?competition .
            
    filter not exists { ?country wdt:P156|wdt:P1366 ?o }
            
    ?competition wdt:P3450 wd:Q1968 .
    
    ?driver <http://schema.org/name> ?driverName .
    ?country <http://schema.org/name> ?countryName .
} group by ?countryName ?driver ?driverName
order by ?countryName desc(?numPartecipations)
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Brazil'), ('driver', 'http://www.wikidata.org/entity/Q10490'), ('driverName', 'Ayrton Senna'), ('numPartecipations', '11')]
[('countryName', 'Canada'), ('driver', 'http://www.wikidata.org/entity/Q172285'), ('driverName', 'Eppie Wietzes'), ('numPartecipations', '2')]
[('countryName', 'Germany'), ('driver', 'http://www.wikidata.org/entity/Q9671'), ('driverName', 'Michael Schumacher'), ('numPartecipations', '7')]
[('countryName', 'Spain'), ('driver', 'http://www.wikidata.org/entity/Q10514'), ('driverName', 'Fernando Alonso'), ('numPartecipations', '15')]
[('countryName', 'Spain'), ('driver', 'http://www.wikidata.org/entity/Q173379'), ('driverName', 'Luis Pérez-Sala'), ('numPartecipations', '2')]
[('countryName', 'Thailand'), ('driver', 'http://www.wikidata.org/entity/Q16149172'), ('driverName', 'Alexander Albon'), ('numPartecipations', '1')]
[('countryName', 'United Kingdom'), ('driver', 'http://www.wikidata.org/entity/Q16149172'), ('driverName', 'Alexander Albon

7

In [18]:
queryString = """
select ?countryName ?team ?teamName (count(distinct ?competition) as ?numPartecipations) where {
    ?team wdt:P31 wd:Q10497835 ;
          wdt:P17/(wdt:P156*|wdt:P1366*|^wdt:P1365*) ?country .
    filter not exists { ?country wdt:P156|wdt:P1366 ?o }
    
    ?driver wdt:P106 wd:Q10841764 ;
            wdt:P54 ?team ;
            wdt:P1344 ?competition .
            
    ?competition wdt:P3450 wd:Q1968 .
    
    ?team <http://schema.org/name> ?teamName .
    ?country <http://schema.org/name> ?countryName .
} group by ?countryName ?team ?teamName
order by ?countryName desc(?numPartecipations)
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Australia'), ('team', 'http://www.wikidata.org/entity/Q173103'), ('teamName', 'Brabham'), ('numPartecipations', '2')]
[('countryName', 'France'), ('team', 'http://www.wikidata.org/entity/Q172722'), ('teamName', 'Renault F1 Team'), ('numPartecipations', '15')]
[('countryName', 'Italy'), ('team', 'http://www.wikidata.org/entity/Q169898'), ('teamName', 'Scuderia Ferrari'), ('numPartecipations', '19')]
[('countryName', 'Italy'), ('team', 'http://www.wikidata.org/entity/Q142381'), ('teamName', 'Minardi'), ('numPartecipations', '17')]
[('countryName', 'Italy'), ('team', 'http://www.wikidata.org/entity/Q171329'), ('teamName', 'Benetton Formula'), ('numPartecipations', '7')]
[('countryName', 'Italy'), ('team', 'http://www.wikidata.org/entity/Q141818'), ('teamName', 'Scuderia Toro Rosso'), ('numPartecipations', '1')]
[('countryName', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q172030'), ('teamName', 'McLaren'), ('numPartecipations', '26')]
[('countryNa

13

I also need the BGP for obtaining the list of all countries. I can look for United Kingdom's *instance of* property.

In [19]:
queryString = """
select ?o ?oName where {
    wd:Q145 wdt:P31 ?o .
    
    ?o <http://schema.org/name> ?oName .
}
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q112099'), ('oName', 'island nation')]
[('o', 'http://www.wikidata.org/entity/Q20181813'), ('oName', 'colonial power')]
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('oName', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('oName', 'country')]
[('o', 'http://www.wikidata.org/entity/Q202686'), ('oName', 'Commonwealth realm')]


5

So, the BGP for obtaining the list of countries is:
    
```
?country wdt:P31 wd:Q3624078
```

Merging all together

In [20]:
queryString = """
select distinct ?countryName ?teamName ?maxTeamCompetitions ?driverName ?maxDriverCompetitions where {
    ?originalCountry wdt:P31 wd:Q3624078 .
    
    optional {
        select ?originalCountry ?driver ?driverName ?maxDriverCompetitions where {
            {
                select ?originalCountry (max(?numDriverCompetitions) as ?maxDriverCompetitions) where {
                     {
                        select ?originalCountry ?driver (count(?competition) as ?numDriverCompetitions) where {
                            ?driver wdt:P106 wd:Q10841764 ;
                                    wdt:P27 ?originalCountry ;
                                    wdt:P1344 ?competition .

                            ?competition wdt:P3450 wd:Q1968 .
                        } group by ?originalCountry ?driver
                    }
                }
            }
            .
            {
                select ?originalCountry ?driver (count(?competition) as ?numDriverCompetitions) where {
                    ?driver wdt:P106 wd:Q10841764 ;
                            wdt:P27 ?originalCountry ;
                            wdt:P1344 ?competition .

                    ?competition wdt:P3450 wd:Q1968 .
                } group by ?originalCountry ?driver
            }
            
            filter (?numDriverCompetitions = ?maxDriverCompetitions)
            ?driver <http://schema.org/name> ?driverName .
        }
    }
    optional {
        select ?originalCountry ?team ?teamName ?maxTeamCompetitions where {
            {
                select ?originalCountry (max(?numTeamCompetitions) as ?maxTeamCompetitions) where {
                     {
                        select ?originalCountry ?team (count(?competition) as ?numTeamCompetitions) where {
                            ?team wdt:P31 wd:Q10497835 ;
                                  wdt:P17 ?originalCountry .

                            ?driver wdt:P106 wd:Q10841764 ;
                                    wdt:P54 ?team ;
                                    wdt:P1344 ?competition .

                            ?competition wdt:P3450 wd:Q1968 .
                        } group by ?originalCountry ?team
                    }
                }
            }
            .
            {
                select ?originalCountry ?team (count(?competition) as ?numTeamCompetitions) where {
                    ?team wdt:P31 wd:Q10497835 ;
                          wdt:P17 ?originalCountry .

                    ?driver wdt:P106 wd:Q10841764 ;
                            wdt:P54 ?team ;
                            wdt:P1344 ?competition .

                    ?competition wdt:P3450 wd:Q1968 .
                } group by ?originalCountry ?team
            }
            
            filter (?numTeamCompetitions = ?maxTeamCompetitions) .
            ?team <http://schema.org/name> ?teamName .
        }
    } .
       
    ?originalCountry wdt:P156*|wdt:P1366*|^wdt:P1365* ?country .
    filter not exists { ?originalCountry wdt:P156|wdt:P1366 ?o }
    ?country <http://schema.org/name> ?countryName .
    
    filter (?maxTeamCompetitions > 0 || ?maxDriverCompetitions > 0)
} order by desc(?maxTeamCompetitions) desc(?maxDriverCompetitions)
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'United Kingdom'), ('teamName', 'McLaren'), ('maxTeamCompetitions', '26'), ('driverName', 'Alexander Albon'), ('maxDriverCompetitions', '1')]
[('countryName', 'Italy'), ('teamName', 'Scuderia Ferrari'), ('maxTeamCompetitions', '22')]
[('countryName', 'France'), ('teamName', 'Renault F1 Team'), ('maxTeamCompetitions', '15')]
[('countryName', 'Australia'), ('teamName', 'Brabham'), ('maxTeamCompetitions', '2')]
[('countryName', 'Spain'), ('driverName', 'Fernando Alonso'), ('maxDriverCompetitions', '15')]
[('countryName', 'Brazil'), ('driverName', 'Ayrton Senna'), ('maxDriverCompetitions', '11')]
[('countryName', 'Germany'), ('driverName', 'Michael Schumacher'), ('maxDriverCompetitions', '7')]
[('countryName', 'Canada'), ('driverName', 'Eppie Wietzes'), ('maxDriverCompetitions', '2')]
[('countryName', 'Thailand'), ('driverName', 'Alexander Albon'), ('maxDriverCompetitions', '1')]
[('countryName', 'Sisophon Province'), ('driverName', 'Alexander Albon'), ('maxDriverC

10